##### 위치로만 인자를 지정하게 하거나 키워드로만 인자를 지정하게 해서   함수 호출을 명확하게 만들라

- 키워드를 사용해 인자를 넘기는 기능은 파이썬 함수의 강력한 기능이다.
- 키워드 인자의 유연성을 활용하면 여러분의 코드를 처음 읽는 사람도 더 명확하게 용례를 이해할 수 있는 함수를 작성할 수 있다.

In [2]:
# 예를들어 한 숫자를 다른 숫자로 나눌 때 세심하게 주의를 기울여야 하는 특별한 경우가 있는지 알고 싶다고 하자.
# zerodivisionerror 예외를 무시하고 무한대를 반환하고 싶고
# OverflowError 예외를 무시하고 대신 0을 반환하고 싶다.
def safe_division(number, divisor,
                  ignore_overflow,
                  ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise


In [3]:
#소수점이 무한히 많을 때 0을 반환.
result = safe_division(1.0, 10**500, True, False)
print(result)
# 나누지 못하는 상황에 int값을 반환
result = safe_division(1.0, 0, False, True)
print(result)


0
inf


- 문제는 어떤 예외를 무시할지 결정하는 두 불 변수의 휘치를 혼동하기 쉽다는 것이다.
- 추적하기 힘든 버그가 생길 수 있다.
- 가독성을 키우는 방법은 키워드 인자를 사용하는 것이다.

In [14]:
def safe_division_b(number, divisor,
                    ignore_overflow=False,       # 변경
                    ignore_zero_division=False): # 변경
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [15]:
# 호출할 때 무시할 예외를 정하는 플래그를 설정할 수 있다.
result = safe_division_b(1.0, 10**500, ignore_overflow=True)
print(result)

result = safe_division_b(1.0, 0, ignore_zero_division=True)
print(result)

0
inf


In [6]:
# 이런 식으로 키워드 인자를 사용하는 것이 선택적 사항이다.
#여전히 위치 인자를 예전방식으로 함수를 호출 할 수 있다.
assert safe_division_b(1.0, 10**500, True, False) == 0

###### 복잡한 함수의 경우 호출자가 키워드만 사용하는 인자를 통해 의도를 명확히 밝히도록 요구하는 편이 좋다.
- 키워드만 사용하는 인자는 키워드를 반드시 사용해 지정해야하며
- 절대 위치를 기반으로는 지정할 수 없다.

In [10]:
# safe_division  함수가 키워드만 사용하는 인자만 받도록 만든 코드
# 목록의 * 기호는 위치 인자의 마지막과 키워드만 사용하는 인자의 시작을 구분
def safe_division_c(number, divisor,         # 변경
                    ignore_overflow=False
                ,*,ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [11]:
# 호출하면서 키워드 인자를 사용해야하는데 위치 인자를 사용하면 작동이 잘 안된다.
safe_division_c(1.0, 10**500, True,ignore_zero_division=False)

0

In [4]:
safe_division_c(1.0, 10**500, ignore_zero_division=True,ignore_overflow=False)

OverflowError: int too large to convert to float

In [10]:
#아래와 같이 작동하면 된다.
result = safe_division_c(1.0, 0, ignore_zero_division=True)
assert result == float('inf')

try:
    result = safe_division_c(1.0, 0)
except ZeroDivisionError:
    pass # 예상대로 작동함
print(result)

inf


- 하지만 safe_division_c 함수도 문제가 있다.
- 호출하는 쪽에서 함수 맨앞 두 필수 인자 (number, divisor)를 호출하면서 위치와 키워드를 혼용할 수 있다.


In [16]:
assert safe_division_c(number=2, divisor=5) == 0.4
assert safe_division_c(divisor=5, number=2) == 0.4
assert safe_division_c(2, divisor=5) == 0.4

In [17]:
# 요구 사항 변경으로 원하는 스타일이 바뀌어서 맨 앞의 두 인자 이름을 변경할 수 있다.
def safe_division_c(numerator, denominator, *,    # 변경
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [18]:
#별거아닌것같지만 과거 인자 키워드 호출 코드는 다 깨진다.
safe_division_c(1.0, 10**500, True, False)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

- 위 같은 경우가 더 큰 문제다.
- 이 함수의 명시적 인터페이스에 number와 divisor를 포함시키는 것을 의도하지 않았기 때문이다.


#### 위치로만 지정하는 인자
- 반드시 위치만 사용해 인자를 지정해야 하고 키워드 인자로는 쓸 수 없다.

In [26]:
# safe)division 함수는 처음 두 필수 인자를 위치로만 지정하는 인자로 지정한다.
# 인자 목록의 / 기호는 위치로만 지정하는 인자의 끝을 표시한다.

#
def safe_division_d(numerator, denominator, /,*, # 변경
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
assert safe_division_d(2, 5) == 0.4

In [18]:
print(safe_division_d(2, 5))

0.4


In [19]:
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
safe_division_d(2, 5)
safe_division_d(numerator=2, denominator=5)

TypeError: safe_division_d() got some positional-only arguments passed as keyword arguments: 'numerator, denominator'

##### 이제 safe_division_d 함수 정의에서 맨 앞의 두 필수 인자는 호출하는 쪽과 분리됐다.
- 내가 파라미터를 변경하더라도 망가지지 않는다

키워드로만 지정하는 인자와 위치로만 지정하는 인자로 인해 생기는 효과
- 인자 목록에서 /와 *기호 사이에 있는 모든 파라미터는 위치를 사용해 전달할 수 있고 이름을 키워드로 사용해 전달할 수도 있다.

In [23]:
# API 스타일이나 필요에 따라 두 인자 전달방식을 모두 사용하면 
# 가독성을 높이고 잡음도 줄일 수 있다.
# 예를 들면 다음 코드에서는 결과를 표시할 때 
# 얼마나 많은 자릿수를 사용할지 결정하고자 반올림할 위치를 지정하는 인자를 
# safe_division에 추가한다.
def safe_division_e(numerator, denominator, /,
                    ndigits=10, *,                 # 변경
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denominator         # 변경
        return round(fraction, ndigits)            # 변경
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise


SyntaxError: invalid syntax (<ipython-input-23-3b9a5f04b2a0>, line 6)

In [24]:
#여러방식의 호출
result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)
print(result)

result = safe_division_e(22, 7, ndigits=2)
print(result)

NameError: name 'safe_division_e' is not defined